# Cousera Applied Data Science Capstone project week 3

I will begin by Importing the necessary libraries. Then I will use Beautiful soup to scrape the table off Wikipedia and add it to an empty list called "GTA". Note: The common method used in scraping table is to find the class"table" in the html, but for me this returned nothing. I did a little search and discovered that "tbody" was a better way to locate the table. I also used the "html.parser" instead of the "lxml", because the lxml kept scraping only the html tags. no table content. 

In [1]:
import bs4 as bs
import urllib.request
import pandas as pd

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.tbody
table_rows = table.find_all('tr')

GTA =[]
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        GTA.append(row)


Now, that the data is in a list format in the set "GTA", we can tabulate it by putting it in the pandas data frame. The column names are also specified. the i print out the first 12 rows.

In [2]:
df = pd.DataFrame(GTA, columns=["Postcode", 'Borough', 'Neighbourhood']) 
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Listing out the number of (rows, columns)

In [3]:
df.shape

(289, 3)

In [4]:
df.describe()

,Postcode,Borough,Neighbourhood
count,289,289,289
unique,180,12,210
top,M8Y,Not assigned,Not assigned
freq,8,77,78


Dropping rows where Borough is not assigned. Then confirming that they have  been droppped

In [5]:
df = df[df.Borough != "Not assigned"]
df.describe()

,Postcode,Borough,Neighbourhood
count,212,212,212
unique,103,11,210
top,M9V,Etobicoke,St. James Town
freq,8,45,2


Some Postcodes have the same Borough but different Neighbourhoods so I grouped and separated them with a comma. "as_index" has to set to false, if you want to keep the numeric index. Otherwise the columns stated in the bracket will become  the index of the table. "as_index" is set to true by default.

In [6]:
df=df.groupby(['Postcode', "Borough",], as_index=False, sort=False).agg(",".join)
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Finally, I check if there are any  "Not assigned" values in the Neighbourhood column.

In [8]:
df.loc[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Not assigned





Since there is only one row with this value. I choose to directly replace it with the Borough name

In [9]:
df.Neighbourhood= df.Neighbourhood.replace('Not assigned', 'Queens Park')

I then try to locate the "Not assigned" value  and find it is not there.

In [10]:
df.loc[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood


Then I summarize the number of rows and columns.

In [11]:
df.shape

(103, 3)

### The end